In [1]:
import pandas as pd

df = pd.read_csv('../input/shoppe-web-infer-csv/test.csv')

if df.shape[0] != 3:

    !pip install efficientnet -q
    import os

    import efficientnet.tfkeras as efn
    import numpy as np
    import pandas as pd
    import tensorflow as tf

    def auto_select_accelerator():
        try:
            tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
            tf.config.experimental_connect_to_cluster(tpu)
            tf.tpu.experimental.initialize_tpu_system(tpu)
            strategy = tf.distribute.experimental.TPUStrategy(tpu)
            print("Running on TPU:", tpu.master())
        except ValueError:
            strategy = tf.distribute.get_strategy()
        print(f"Running on {strategy.num_replicas_in_sync} replicas")

        return strategy


    def build_decoder(with_labels=True, target_size=(300, 300), ext='jpg'):
        def decode(path):
            file_bytes = tf.io.read_file(path)
            if ext == 'png':
                img = tf.image.decode_png(file_bytes, channels=3)
            elif ext in ['jpg', 'jpeg']:
                img = tf.image.decode_jpeg(file_bytes, channels=3)
            else:
                raise ValueError("Image extension not supported")

            img = tf.cast(img, tf.float32) / 255.0
            img = tf.image.resize(img, target_size)

            return img

        def decode_with_labels(path, label):
            return decode(path), label

        return decode_with_labels if with_labels else decode


    def build_augmenter(with_labels=True):
        def augment(img):
            img = tf.image.random_flip_left_right(img)
            img = tf.image.random_flip_up_down(img)
            return img

        def augment_with_labels(img, label):
            return augment(img), label

        return augment_with_labels if with_labels else augment


    def build_dataset(paths, labels=None, bsize=32, cache=True,
                      decode_fn=None, augment_fn=None,
                      augment=True, repeat=True, shuffle=1024, 
                      cache_dir=""):
        if cache_dir != "" and cache is True:
            os.makedirs(cache_dir, exist_ok=True)

        if decode_fn is None:
            decode_fn = build_decoder(labels is not None)

        if augment_fn is None:
            augment_fn = build_augmenter(labels is not None)

        AUTO = tf.data.experimental.AUTOTUNE
        slices = paths if labels is None else (paths, labels)

        dset = tf.data.Dataset.from_tensor_slices(slices)
        dset = dset.map(decode_fn, num_parallel_calls=AUTO)
        dset = dset.cache(cache_dir) if cache else dset
        dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
        dset = dset.repeat() if repeat else dset
        dset = dset.shuffle(shuffle) if shuffle else dset
        dset = dset.batch(bsize).prefetch(AUTO)

        return dset

    COMPETITION_NAME = "shoppe-web-csv"
    strategy = auto_select_accelerator()
    BATCH_SIZE = strategy.num_replicas_in_sync * 1024

    IMSIZE = (224, 240, 260, 300, 380, 456, 528, 600)

    load_dir = f"/kaggle/input/{COMPETITION_NAME}/"
    sub_df = pd.read_csv('../input/shoppe-web-infer-csv/test.csv')
    test_paths = sub_df['path'] 

    sub_df['clothers'] = 0
    sub_df['other'] = 0
    

    label_cols = sub_df.columns[8:]

    test_decoder = build_decoder(with_labels=False, target_size=(IMSIZE[0], IMSIZE[0]))
    dtest = build_dataset(
        test_paths, bsize=BATCH_SIZE, repeat=False, 
        shuffle=False, augment=False, cache=False,
        decode_fn=test_decoder
    )

    with strategy.scope():
        model = tf.keras.models.load_model(
            '../input/shopee-2-class-train-efnb0-2/modelb00.h5'
        )

    model.summary()

    sub_df[label_cols] = model.predict(dtest, verbose=1)
    df = sub_df.copy()
    del sub_df

    df_other = df[df['other'] >= 0.5]
    df_clothes = df[df['other'] < 0.5]
    df_other_len = df_other.shape[0]
    df_other = df_other.append(df_clothes).reset_index(drop=True)
    df = df_other.copy()
    del df_other, df_clothes
else:
    df['clothes'] = [0,1,0]
    df['other'] = [1,0,1]    
    df_other = df[df['other'] >= 0.5]
    df_clothes = df[df['other'] < 0.5]
    df_other = df_other.append(df_clothes).reset_index(drop=True)
    df = df_other.copy()   
    df_other_len = 2

Running on 1 replicas


2022-05-01 16:36:53.163072: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-01 16:36:53.283012: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-01 16:36:53.284232: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-01 16:36:53.286263: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b0 (Functional) (None, 7, 7, 1280)        4049564   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 2562      
Total params: 4,052,126
Trainable params: 4,010,110
Non-trainable params: 42,016
_________________________________________________________________


2022-05-01 16:37:00.709525: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
Cleanup called...
Cleanup called...
Cleanup called...
2022-05-01 16:37:15.899209: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005
Cleanup called...
2022-05-01 16:37:22.291896: W tensorflow/core/kernels/gpu_utils.cc:49] Failed to allocate memory for convolution redzone checking; skipping this check. This is benign and only means that we won't check cudnn for out-of-bounds reads and writes. This message will only be printed once.


  1/135 [..............................] - ETA: 52:54

...

132/135 [============================>.] - ETA: 28s

Cleanup called...


133/135 [============================>.] - ETA: 18s

Cleanup called...


135/135 [==============================] - 1291s 9s/step


In [2]:
from numba import cuda
import torch
cuda.select_device(0)
cuda.close()
cuda.select_device(0)

<weakproxy at 0x7fc2dd4ecbf0 to Device at 0x7fc355e0bad0>

In [3]:
import sys, os
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')

import numpy as np 
import pandas as pd 

import math, random
import cv2
import timm
from tqdm import tqdm 

import albumentations as A 
from albumentations.pytorch.transforms import ToTensorV2

import torch 
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch import nn
import torch.nn.functional as F 
import torchvision.models as models
from torch.nn import Parameter

import gc
import cudf, cuml, cupy
from cuml.feature_extraction.text import TfidfVectorizer
from cuml.neighbors import NearestNeighbors

import copy
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import transformers
from transformers import (BertTokenizer, BertModel,
                          DistilBertTokenizer, DistilBertModel)

In [4]:
COMPUTE_CV = False
SAVE_IMGEMBEDDING = False
EFF_B5 = False
ECA_NFNET_L0 = False
MODEL_TESTING_NFNET = False
MY_NFNET = True
BERT = True
DISTILBERT = False
SAVE_DISTILBERT = False # You need to enable internet to download pretrained model
EMBEDDING34_TH = 0.30

#df = pd.read_csv('../input/shopee-product-matching/test.csv')
if len(df)>3: COMPUTE_CV = False
if COMPUTE_CV: 
    print('this submission notebook will compute CV score but commit notebook will not')
else:
    print('this submission notebook will only be used to submit result')

this submission notebook will only be used to submit result


In [5]:
class CFG:
    
    img_size = 512
    fc_dim = 512
    batch_size = 20
    seed = 2020
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    classes = 11014
    classes_other = 11014 - 3282
    classes_clothes = 3282
        
    model_name = 'tf_efficientnet_b5_ns'
    model_name2 = 'eca_nfnet_l0'
    model_name3 = 'efficientnet_b3'
    model_name5 = 'eca_nfnet_l0'
    model_name6 = 'eca_nfnet_l0'
    
    model_path = '../input/shopee-pytorch-models/arcface_512x512_eff_b5_.pt'
    if MODEL_TESTING_NFNET or MY_NFNET:
        model_path2 = '../input/shopee-price-match-guarantee-embeddings/arcface_512x512_nfnet_l0(mish)_b24_15.pt'
    else:
        model_path2 = '../input/shopee-pytorch-models/arcface_512x512_nfnet_l0 (mish).pt'
    model_path3 = '../input/shopee-pytorch-models/arcface_512x512_eff_b3.pt'
    model_path5 = '../input/other-eca-nfnet-l0-training-21ep/arcface_512x512_nfnet_l0(mish)19.pt'
    model_path6 = '../input/clothes-eca-nfnet-l0-training-30ep/arcface_512x512_nfnet_l0(mish)28.pt'
    scale = 30 
    margin = 0.5

In [6]:
def read_dataset(COMPUTE_CV,df = df):
    
    if COMPUTE_CV:
        df_cu = cudf.DataFrame(df)
        image_paths = df['path']
    
    else:
        df_cu = cudf.DataFrame(df)
        image_paths = df['path']

    return df, df_cu, image_paths
def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_torch(CFG.seed)

In [7]:
def combine_predictions(row):
    x = np.concatenate([row['image_predictions'], row['text_predictions'], row['phash_predictions']])
    return ' '.join( np.unique(x))
def combine_for_cv(row):
    x = np.concatenate([row['image_predictions'], row['text_predictions'], row['phash_predictions']])
    return np.unique(x)
def combine_predictions_BERT(row):
    x = np.concatenate([row['image_predictions'], row['text_predictions'], row['phash_predictions'], row['text_predictions_BERT']])
    return ' '.join( np.unique(x))
def combine_for_cv_BERT(row):
    x = np.concatenate([row['image_predictions'], row['text_predictions'], row['phash_predictions'], row['text_predictions_BERT']])
    return np.unique(x)

In [8]:
def getMetric(col):
    def f1score(row):
        n = len(np.intersect1d(row.target, row[col]))
        return 2*n / (len(row.target) + len(row[col]))
    return f1score

In [9]:
def get_test_transforms():

    return A.Compose(
        [
            A.Resize(CFG.img_size,CFG.img_size,always_apply=True),
            A.Normalize(),
        ToTensorV2(p=1.0)
        ]
    )

In [10]:
class ShopeeDataset(Dataset):
    def __init__(self, image_paths, transforms=None):

        self.image_paths = image_paths
        self.augmentations = transforms

    def __len__(self):
        return self.image_paths.shape[0]

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.augmentations:
            augmented = self.augmentations(image=image)
            image = augmented['image']       
    
        return image,torch.tensor(1)

In [11]:
class ArcMarginProduct(nn.Module):
    def __init__(self, in_features, out_features, scale=30.0, margin=0.50, easy_margin=False, ls_eps=0.0):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.scale = scale
        self.margin = margin
        self.ls_eps = ls_eps  # label smoothing
        self.weight = Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(margin)
        self.sin_m = math.sin(margin)
        self.th = math.cos(math.pi - margin)
        self.mm = math.sin(math.pi - margin) * margin

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size(), device='cuda')
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.out_features
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.scale

        return output
    

class ShopeeModel(nn.Module):

    def __init__(
        self,
        n_classes = CFG.classes,
        model_name = CFG.model_name,
        fc_dim = 512,
        margin = CFG.margin,
        scale = CFG.scale,
        use_fc = False,
        pretrained = False):


        super(ShopeeModel,self).__init__()
        print('Building Model Backbone for {} model'.format(model_name))

        self.backbone = timm.create_model(model_name, pretrained=pretrained)

        if model_name == 'resnext50_32x4d':
            final_in_features = self.backbone.fc.in_features
            self.backbone.fc = nn.Identity()
            self.backbone.global_pool = nn.Identity()

        elif model_name == 'efficientnet_b3':
            final_in_features = self.backbone.classifier.in_features
            self.backbone.classifier = nn.Identity()
            self.backbone.global_pool = nn.Identity()

        elif model_name == 'tf_efficientnet_b5_ns':
            final_in_features = self.backbone.classifier.in_features
            self.backbone.classifier = nn.Identity()
            self.backbone.global_pool = nn.Identity()
        
        elif model_name == 'nfnet_f3':
            final_in_features = self.backbone.head.fc.in_features
            self.backbone.head.fc = nn.Identity()
            self.backbone.head.global_pool = nn.Identity()

        self.pooling =  nn.AdaptiveAvgPool2d(1)

        self.use_fc = use_fc

        self.dropout = nn.Dropout(p=0.0)
        self.fc = nn.Linear(final_in_features, fc_dim)
        self.bn = nn.BatchNorm1d(fc_dim)
        self._init_params()
        final_in_features = fc_dim

        self.final = ArcMarginProduct(
            final_in_features,
            n_classes,
            scale = scale,
            margin = margin,
            easy_margin = False,
            ls_eps = 0.0
        )

    def _init_params(self):
        nn.init.xavier_normal_(self.fc.weight)
        nn.init.constant_(self.fc.bias, 0)
        nn.init.constant_(self.bn.weight, 1)
        nn.init.constant_(self.bn.bias, 0)

    def forward(self, image, label):
        feature = self.extract_feat(image)
        #logits = self.final(feature,label)
        return feature

    def extract_feat(self, x):
        batch_size = x.shape[0]
        x = self.backbone(x)
        x = self.pooling(x).view(batch_size, -1)

        if self.use_fc:
            x = self.dropout(x)
            x = self.fc(x)
            x = self.bn(x)
        return x
    
    
class ShopeeModel2(nn.Module):

    def __init__(
        self,
        n_classes = CFG.classes,
        model_name = CFG.model_name2,
        fc_dim = 512,
        margin = CFG.margin,
        scale = CFG.scale,
        use_fc = True,
        pretrained = False):


        super(ShopeeModel2,self).__init__()
        print('Building Model Backbone for {} model'.format(model_name))

        self.backbone = timm.create_model(model_name, pretrained=pretrained)

        if model_name == 'resnext50_32x4d':
            final_in_features = self.backbone.fc.in_features
            self.backbone.fc = nn.Identity()
            self.backbone.global_pool = nn.Identity()

        elif model_name == 'efficientnet_b3':
            final_in_features = self.backbone.classifier.in_features
            self.backbone.classifier = nn.Identity()
            self.backbone.global_pool = nn.Identity()

        elif model_name == 'tf_efficientnet_b5_ns':
            final_in_features = self.backbone.classifier.in_features
            self.backbone.classifier = nn.Identity()
            self.backbone.global_pool = nn.Identity()
        
        elif model_name == 'eca_nfnet_l0':
            final_in_features = self.backbone.head.fc.in_features
            self.backbone.head.fc = nn.Identity()
            self.backbone.head.global_pool = nn.Identity()

        self.pooling =  nn.AdaptiveAvgPool2d(1)

        self.use_fc = use_fc

        self.dropout = nn.Dropout(p=0.0)
        self.fc = nn.Linear(final_in_features, fc_dim)
        self.bn = nn.BatchNorm1d(fc_dim)
        self._init_params()
        final_in_features = fc_dim

        self.final = ArcMarginProduct(
            final_in_features,
            n_classes,
            scale = scale,
            margin = margin,
            easy_margin = False,
            ls_eps = 0.0
        )

    def _init_params(self):
        nn.init.xavier_normal_(self.fc.weight)
        nn.init.constant_(self.fc.bias, 0)
        nn.init.constant_(self.bn.weight, 1)
        nn.init.constant_(self.bn.bias, 0)

    def forward(self, image, label):
        feature = self.extract_feat(image)
        #logits = self.final(feature,label)
        return feature

    def extract_feat(self, x):
        batch_size = x.shape[0]
        x = self.backbone(x)
        x = self.pooling(x).view(batch_size, -1)

        if self.use_fc:
            x = self.dropout(x)
            x = self.fc(x)
            x = self.bn(x)
        return x
    
    
class ShopeeModel3(nn.Module):

    def __init__(
        self, 
        model_name = CFG.model_name3,
        n_classes = CFG.classes,
        fc_dim = 512,
        margin = CFG.margin,
        scale = CFG.scale,
        use_fc = True,
        pretrained = False):


        super(ShopeeModel3,self).__init__()
        print('Building Model Backbone for {} model'.format(model_name))

        self.backbone = timm.create_model(model_name, pretrained=pretrained)

        if model_name == 'resnext50_32x4d':
            final_in_features = self.backbone.fc.in_features
            self.backbone.fc = nn.Identity()
            self.backbone.global_pool = nn.Identity()

        elif model_name == 'efficientnet_b3':
            final_in_features = self.backbone.classifier.in_features
            self.backbone.classifier = nn.Identity()
            self.backbone.global_pool = nn.Identity()

        elif model_name == 'tf_efficientnet_b5_ns':
            final_in_features = self.backbone.classifier.in_features
            self.backbone.classifier = nn.Identity()
            self.backbone.global_pool = nn.Identity()
        
        elif model_name == 'eca_nfnet_l0':
            final_in_features = self.backbone.head.fc.in_features
            self.backbone.head.fc = nn.Identity()
            self.backbone.head.global_pool = nn.Identity()

        self.pooling =  nn.AdaptiveAvgPool2d(1)

        self.use_fc = use_fc

        self.dropout = nn.Dropout(p=0.0)
        self.fc = nn.Linear(final_in_features, fc_dim)
        self.bn = nn.BatchNorm1d(fc_dim)
        self._init_params()
        final_in_features = fc_dim

        self.final = ArcMarginProduct(
            final_in_features,
            n_classes,
            scale = scale,
            margin = margin,
            easy_margin = False,
            ls_eps = 0.0
        )

    def _init_params(self):
        nn.init.xavier_normal_(self.fc.weight)
        nn.init.constant_(self.fc.bias, 0)
        nn.init.constant_(self.bn.weight, 1)
        nn.init.constant_(self.bn.bias, 0)

    def forward(self, image, label):
        feature = self.extract_feat(image)
        #logits = self.final(feature,label)
        return feature

    def extract_feat(self, x):
        batch_size = x.shape[0]
        x = self.backbone(x)
        x = self.pooling(x).view(batch_size, -1)

        if self.use_fc:
            x = self.dropout(x)
            x = self.fc(x)
            x = self.bn(x)
        return x
    
class ShopeeModel5(nn.Module):

    def __init__(
        self,
        n_classes = CFG.classes_other,
        model_name = CFG.model_name5,
        fc_dim = 512,
        margin = CFG.margin,
        scale = CFG.scale,
        use_fc = True,
        pretrained = False):


        super(ShopeeModel5,self).__init__()
        print('Building Model Backbone for {} model'.format(model_name))

        self.backbone = timm.create_model(model_name, pretrained=pretrained)

        if model_name == 'resnext50_32x4d':
            final_in_features = self.backbone.fc.in_features
            self.backbone.fc = nn.Identity()
            self.backbone.global_pool = nn.Identity()

        elif model_name == 'efficientnet_b3':
            final_in_features = self.backbone.classifier.in_features
            self.backbone.classifier = nn.Identity()
            self.backbone.global_pool = nn.Identity()

        elif model_name == 'tf_efficientnet_b5_ns':
            final_in_features = self.backbone.classifier.in_features
            self.backbone.classifier = nn.Identity()
            self.backbone.global_pool = nn.Identity()
        
        elif model_name == 'eca_nfnet_l0':
            final_in_features = self.backbone.head.fc.in_features
            self.backbone.head.fc = nn.Identity()
            self.backbone.head.global_pool = nn.Identity()

        self.pooling =  nn.AdaptiveAvgPool2d(1)

        self.use_fc = use_fc

        self.dropout = nn.Dropout(p=0.0)
        self.fc = nn.Linear(final_in_features, fc_dim)
        self.bn = nn.BatchNorm1d(fc_dim)
        self._init_params()
        final_in_features = fc_dim

        self.final = ArcMarginProduct(
            final_in_features,
            n_classes,
            scale = scale,
            margin = margin,
            easy_margin = False,
            ls_eps = 0.0
        )

    def _init_params(self):
        nn.init.xavier_normal_(self.fc.weight)
        nn.init.constant_(self.fc.bias, 0)
        nn.init.constant_(self.bn.weight, 1)
        nn.init.constant_(self.bn.bias, 0)

    def forward(self, image, label):
        feature = self.extract_feat(image)
        #logits = self.final(feature,label)
        return feature

    def extract_feat(self, x):
        batch_size = x.shape[0]
        x = self.backbone(x)
        x = self.pooling(x).view(batch_size, -1)

        if self.use_fc:
            x = self.dropout(x)
            x = self.fc(x)
            x = self.bn(x)
        return x
    
class ShopeeModel6(nn.Module):

    def __init__(
        self,
        n_classes = CFG.classes_clothes,
        model_name = CFG.model_name6,
        fc_dim = 512,
        margin = CFG.margin,
        scale = CFG.scale,
        use_fc = True,
        pretrained = False):


        super(ShopeeModel6,self).__init__()
        print('Building Model Backbone for {} model'.format(model_name))

        self.backbone = timm.create_model(model_name, pretrained=pretrained)

        if model_name == 'resnext50_32x4d':
            final_in_features = self.backbone.fc.in_features
            self.backbone.fc = nn.Identity()
            self.backbone.global_pool = nn.Identity()

        elif model_name == 'efficientnet_b3':
            final_in_features = self.backbone.classifier.in_features
            self.backbone.classifier = nn.Identity()
            self.backbone.global_pool = nn.Identity()

        elif model_name == 'tf_efficientnet_b5_ns':
            final_in_features = self.backbone.classifier.in_features
            self.backbone.classifier = nn.Identity()
            self.backbone.global_pool = nn.Identity()
        
        elif model_name == 'eca_nfnet_l0':
            final_in_features = self.backbone.head.fc.in_features
            self.backbone.head.fc = nn.Identity()
            self.backbone.head.global_pool = nn.Identity()

        self.pooling =  nn.AdaptiveAvgPool2d(1)

        self.use_fc = use_fc

        self.dropout = nn.Dropout(p=0.0)
        self.fc = nn.Linear(final_in_features, fc_dim)
        self.bn = nn.BatchNorm1d(fc_dim)
        self._init_params()
        final_in_features = fc_dim

        self.final = ArcMarginProduct(
            final_in_features,
            n_classes,
            scale = scale,
            margin = margin,
            easy_margin = False,
            ls_eps = 0.0
        )

    def _init_params(self):
        nn.init.xavier_normal_(self.fc.weight)
        nn.init.constant_(self.fc.bias, 0)
        nn.init.constant_(self.bn.weight, 1)
        nn.init.constant_(self.bn.bias, 0)

    def forward(self, image, label):
        feature = self.extract_feat(image)
        #logits = self.final(feature,label)
        return feature

    def extract_feat(self, x):
        batch_size = x.shape[0]
        x = self.backbone(x)
        x = self.pooling(x).view(batch_size, -1)

        if self.use_fc:
            x = self.dropout(x)
            x = self.fc(x)
            x = self.bn(x)
        return x

In [12]:
# https://www.kaggle.com/parthdhameliya77/pytorch-eca-nfnet-l0-image-tfidf-inference
class Mish_func(torch.autograd.Function):
    
    """from: https://github.com/tyunist/memory_efficient_mish_swish/blob/master/mish.py"""
    
    @staticmethod
    def forward(ctx, i):
        result = i * torch.tanh(F.softplus(i))
        ctx.save_for_backward(i)
        return result

    @staticmethod
    def backward(ctx, grad_output):
        i = ctx.saved_variables[0]
  
        v = 1. + i.exp()
        h = v.log() 
        grad_gh = 1./h.cosh().pow_(2) 

        # Note that grad_hv * grad_vx = sigmoid(x)
        #grad_hv = 1./v  
        #grad_vx = i.exp()
        
        grad_hx = i.sigmoid()

        grad_gx = grad_gh *  grad_hx #grad_hv * grad_vx 
        
        grad_f =  torch.tanh(F.softplus(i)) + i * grad_gx 
        
        return grad_output * grad_f 


class Mish(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
        pass
    def forward(self, input_tensor):
        return Mish_func.apply(input_tensor)


def replace_activations(model, existing_layer, new_layer):
    
    """A function for replacing existing activation layers"""
    
    for name, module in reversed(model._modules.items()):
        if len(list(module.children())) > 0:
            model._modules[name] = replace_activations(module, existing_layer, new_layer)

        if type(module) == existing_layer:
            layer_old = module
            layer_new = new_layer
            model._modules[name] = layer_new
    return model

In [13]:
def get_image_embeddings(image_paths, model_name=CFG.model_name, EFF_B5=EFF_B5, nfnet_only=False):
    
    if EFF_B5 and not nfnet_only:
        embeds = []

        model = ShopeeModel(model_name = model_name)
        model.eval()
        model.load_state_dict(torch.load(CFG.model_path))
        model = model.to(CFG.device)

        image_dataset = ShopeeDataset(image_paths=image_paths,transforms=get_test_transforms())
        image_loader = DataLoader(
            image_dataset,
            batch_size=CFG.batch_size,
            pin_memory=True,
            drop_last=False,
            num_workers=4
        )

        with torch.no_grad():
            for img,label in tqdm(image_loader): 
                img = img.cuda()
                label = label.cuda()
                feat = model(img,label)
                image_embeddings = feat.detach().cpu().numpy()
                embeds.append(image_embeddings)


        del model, image_embeddings
        image_embeddings1 = np.concatenate(embeds)
        print(f'image embeddings1 shape is {image_embeddings1.shape}')
        del embeds
        gc.collect()
        
    else: image_embeddings1 = None
    
    #---
    
    model = ShopeeModel2()
    model.eval()
    model = replace_activations(model, torch.nn.SiLU, Mish())

    model.load_state_dict(torch.load(CFG.model_path2))
    model = model.to(CFG.device)
    
    image_dataset = ShopeeDataset(image_paths=image_paths,transforms=get_test_transforms())
    image_loader = DataLoader(
        image_dataset,
        batch_size=CFG.batch_size,
        pin_memory=True,
        drop_last=False,
        num_workers=4
    )
    
    embeds2 = []
    with torch.no_grad():
        for img,label in tqdm(image_loader): 
            img = img.cuda()
            label = label.cuda()
            feat = model(img,label)
            image_embeddings = feat.detach().cpu().numpy()
            embeds2.append(image_embeddings)
    
    del model
    image_embeddings3 = np.concatenate(embeds2)
    print(f'image embeddings3 shape is {image_embeddings3.shape}')
    del embeds2
    gc.collect()
    
    #---
    if not nfnet_only:
        embeds = []

        model = ShopeeModel3()
        model.eval()
        model.load_state_dict(torch.load(CFG.model_path3))
        model = model.to(CFG.device)


        image_dataset = ShopeeDataset(image_paths=image_paths,transforms=get_test_transforms())
        image_loader = DataLoader(
            image_dataset,
            batch_size=CFG.batch_size,
            pin_memory=True,
            drop_last=False,
            num_workers=4
        )


        with torch.no_grad():
            for img,label in tqdm(image_loader): 
                img = img.cuda()
                label = label.cuda()
                feat = model(img,label)
                image_embeddings = feat.detach().cpu().numpy()
                embeds.append(image_embeddings)


        del model
        image_embeddings4 = np.concatenate(embeds)
        print(f'Our image embeddings shape is {image_embeddings.shape}')
        del embeds
        gc.collect()    

    else: image_embeddings4 = None

    embeds = []
    
    model = ShopeeModel5()
    model.eval()
    model.load_state_dict(torch.load(CFG.model_path5))
    model = model.to(CFG.device)
    

    image_dataset = ShopeeDataset(image_paths=image_paths,transforms=get_test_transforms())
    image_loader = torch.utils.data.DataLoader(
        image_dataset,
        batch_size=CFG.batch_size,
        pin_memory=True,
        drop_last=False,
        num_workers=4
    )
    
    
    with torch.no_grad():
        for img,label in tqdm(image_loader): 
            img = img.cuda()
            label = label.cuda()
            feat = model(img,label)
            image_embeddings = feat.detach().cpu().numpy()
            embeds.append(image_embeddings)
    
    
    del model
    image_embeddings5 = np.concatenate(embeds)
    print(f'Our image embeddings shape is {image_embeddings.shape}')
    del embeds
    gc.collect() 
    
    embeds = []
    
    model = ShopeeModel6()
    model.eval()
    model.load_state_dict(torch.load(CFG.model_path6))
    model = model.to(CFG.device)
    

    image_dataset = ShopeeDataset(image_paths=image_paths,transforms=get_test_transforms())
    image_loader = torch.utils.data.DataLoader(
        image_dataset,
        batch_size=CFG.batch_size,
        pin_memory=True,
        drop_last=False,
        num_workers=4
    )
    
    
    with torch.no_grad():
        for img,label in tqdm(image_loader): 
            img = img.cuda()
            label = label.cuda()
            feat = model(img,label)
            image_embeddings = feat.detach().cpu().numpy()
            embeds.append(image_embeddings)
    
    
    del model
    image_embeddings6 = np.concatenate(embeds)
    print(f'Our image embeddings shape is {image_embeddings.shape}')
    del embeds
    gc.collect() 
    return image_embeddings1, image_embeddings3, image_embeddings4, image_embeddings5, image_embeddings6

In [14]:
# Threshold shifting depending on dataset length
# https://www.kaggle.com/c/shopee-product-matching/discussion/234927
def dataset_th(known_th, known_dataset_len, new_dataset_len):
    return (-2.051562606852219e-06 * (new_dataset_len-known_dataset_len)) + known_th

dataset_th(1.7, 34250, 70000)

1.626656636805033

In [15]:
def get_image_predictions_other(df, embeddings1, embeddings3, embeddings34, 
                          EFF_B5=EFF_B5, ECA_NFNET_L0=ECA_NFNET_L0, predictions34_th=0.36):
    
    if len(df) > 3:
        KNN = 50
    else : 
        KNN = 3
    
    #--
    if EFF_B5:
        model = NearestNeighbors(n_neighbors = KNN)
        model.fit(embeddings1)
        distances, indices = model.kneighbors(embeddings1)

        threshold = 1.7 - 0.2978
        predictions1 = []
        for k in tqdm(range(0, df_other_len)):
            idx = np.where(distances[k,] < threshold)[0]
            ids = indices[k,idx]
            posting_ids = list(df['posting_id'].iloc[ids])
            # for ii in np.arange(1.7-0.2978, (1.7-0.2978)*1.5, 0.04):
                # if ii < (1.7-0.2978)*1.5 and len(posting_ids) <= 1:
                    # idx = np.where(distances[k,] < ii)[0]
                    # ids = indices[k,idx]
                    # posting_ids = list(df['posting_id'].iloc[ids].values)  
            predictions1.append(posting_ids)

        del model, distances, indices, embeddings1
        gc.collect()

    #--
    if ECA_NFNET_L0:
        model = NearestNeighbors(n_neighbors = KNN, metric = 'cosine')
        model.fit(embeddings3)
        distances, indices = model.kneighbors(embeddings3)

        threshold=0.36
        predictions3 = []
        for k in tqdm(range((0, df_other_len))):
            idx = np.where(distances[k,] < threshold)[0]
            ids = indices[k,idx]
            posting_ids = list(df['posting_id'].iloc[ids])
            predictions3.append(posting_ids)

        del model, distances, indices, embeddings3
        gc.collect()    
    #--
    
    model = NearestNeighbors(n_neighbors = KNN, metric = 'cosine')
    model.fit(embeddings34)
    distances, indices = model.kneighbors(embeddings34)
    
    predictions34 = []
    for k in tqdm(range(0, df_other_len)):
        idx = np.where(distances[k,] < predictions34_th)[0]
        ids = indices[k,idx]
        posting_ids = list(df['posting_id'].iloc[ids].values)
        for ii in np.arange(predictions34_th, 1.5, 0.1):
            #print(ii)
            if len(posting_ids) <= 5:
                idx = np.where(distances[k,] < ii)[0]
                ids = indices[k,idx]
                posting_ids = list(df['posting_id'].iloc[ids].values) 
            else:
                break
        predictions34.append(posting_ids)
        
    del model, distances, indices
    gc.collect()
    
    # combine predictions(i.e. image IDs) of all the models & remove the duplicates.
    # we can try & experiment here to combine different models here..
    if EFF_B5 and ECA_NFNET_L0:
        predictions = [list(set(a + c + d)) for a, c, d in zip(predictions1, predictions3, predictions34)]
    elif EFF_B5:
        predictions = [list(set(a + d)) for a, d in zip(predictions1, predictions34)]
    else:
        predictions = predictions34

    
    return predictions

In [16]:
def get_image_predictions_clothes(df, embeddings1, embeddings3, embeddings34, 
                          EFF_B5=EFF_B5, ECA_NFNET_L0=ECA_NFNET_L0, predictions34_th=0.36):
    
    if len(df) > 3:
        KNN = 50
    else : 
        KNN = 3
    
    #--
    if EFF_B5:
        model = NearestNeighbors(n_neighbors = KNN)
        model.fit(embeddings1)
        distances, indices = model.kneighbors(embeddings1)

        threshold = 1.7 - 0.2978
        predictions1 = []
        for k in tqdm(range(df_other_len, df.shape[0])):
            idx = np.where(distances[k,] < threshold)[0]
            ids = indices[k,idx]
            posting_ids = list(df['posting_id'].iloc[ids])
            # for ii in np.arange(1.7-0.2978, (1.7-0.2978)*1.5, 0.04):
                # if ii < (1.7-0.2978)*1.5 and len(posting_ids) <= 1:
                    # idx = np.where(distances[k,] < ii)[0]
                    # ids = indices[k,idx]
                    # posting_ids = list(df['posting_id'].iloc[ids].values)  
            predictions1.append(posting_ids)

        del model, distances, indices, embeddings1
        gc.collect()

    #--
    if ECA_NFNET_L0:
        model = NearestNeighbors(n_neighbors = KNN, metric = 'cosine')
        model.fit(embeddings3)
        distances, indices = model.kneighbors(embeddings3)

        threshold=0.36
        predictions3 = []
        for k in tqdm(range(df_other_len, df.shape[0])):
            idx = np.where(distances[k,] < threshold)[0]
            ids = indices[k,idx]
            posting_ids = list(df['posting_id'].iloc[ids])
            predictions3.append(posting_ids)

        del model, distances, indices, embeddings3
        gc.collect()    
    #--
    
    model = NearestNeighbors(n_neighbors = KNN, metric = 'cosine')
    model.fit(embeddings34)
    distances, indices = model.kneighbors(embeddings34)
    
    predictions34 = []
    for k in tqdm(range(df_other_len, df.shape[0])):
        idx = np.where(distances[k,] < predictions34_th)[0]
        ids = indices[k,idx]
        posting_ids = list(df['posting_id'].iloc[ids].values)
        for ii in np.arange(predictions34_th, 1.5, 0.1):
            #print(ii)
            if len(posting_ids) <= 5:
                idx = np.where(distances[k,] < ii)[0]
                ids = indices[k,idx]
                posting_ids = list(df['posting_id'].iloc[ids].values) 
            else:
                break    
        predictions34.append(posting_ids)
        
    del model, distances, indices
    gc.collect()
    
    # combine predictions(i.e. image IDs) of all the models & remove the duplicates.
    # we can try & experiment here to combine different models here..
    if EFF_B5 and ECA_NFNET_L0:
        predictions = [list(set(a + c + d)) for a, c, d in zip(predictions1, predictions3, predictions34)]
    elif EFF_B5:
        predictions = [list(set(a + d)) for a, d in zip(predictions1, predictions34)]
    else:
        predictions = predictions34

    
    return predictions

In [17]:
df, df_cu, image_paths = read_dataset(COMPUTE_CV)
df.head()

,posting_id,name,image,price,category,sort_by_rank,sales_volume,path,clothers,other
0,girls_clothes0002,台灣現貨 A027 海棉胸墊 內衣胸墊 泳衣胸墊 比基尼胸墊 上薄下厚胸墊 集中胸墊 增厚...,cb40c72c7288c7755f8e26e125910468,29,girls_clothes,2,0,../input/shoppe-web-dataset1/girls_clothes0002...,0.001951,0.998049
1,girls_clothes0005,衝浪繩正宗泰國進口蠶絲蠟線 不退色 可帶著洗澡手作diy材料☆水晶森林手創館☆,1b171686050e6fafee1bb666ba62d76c,30,girls_clothes,5,0,../input/shoppe-web-dataset1/girls_clothes0005...,0.162054,0.837946
2,girls_clothes0007,★ 現貨在台 24H內出貨 ★【伊代】一次性乳貼 透氣乳貼 防凸點貼 防走光 乳暈貼 乳頭貼...,577d9263ad55f154524882f8830a574b,1,girls_clothes,7,0,../input/shoppe-web-dataset1/girls_clothes0007...,0.000038,0.999962
3,girls_clothes0008,🥀旺斯女孩🥀歐美一次性胸貼 乳貼 不織布材質 隱形透氣乳貼 防凸點貼 防走光 防凸點 無痕...,25be8b3cb3fe64a58973eb4d1a6aef0c,3,girls_clothes,8,0,../input/shoppe-web-dataset1/girls_clothes0008...,0.149889,0.850111
4,girls_clothes0013,【A】《3排3扣》特價＊台灣現貨免縫內衣延長背扣／內衣扣延長扣加長扣／內衣太緊背勾排扣胸罩文...,23cd7a3db17ec335babe4436a4638358,4,girls_clothes,13,0,../input/shoppe-web-dataset1/girls_clothes0013...,0.183548,0.816452


In [18]:
if not COMPUTE_CV or SAVE_IMGEMBEDDING:
    image_embeddings1, image_embeddings3, image_embeddings4, image_embeddings5, image_embeddings6 = get_image_embeddings(image_paths.values, nfnet_only=MODEL_TESTING_NFNET if COMPUTE_CV else COMPUTE_CV)
    if SAVE_IMGEMBEDDING and not MODEL_TESTING_NFNET: 
        np.savetxt('tf_efficientnet_b5_ns.csv', image_embeddings1, delimiter=',')
        np.savetxt('eca_nfnet_l0.csv', image_embeddings3, delimiter=',')
        np.savetxt('efficientnet_b3.csv', image_embeddings4, delimiter=',')
    elif COMPUTE_CV and MODEL_TESTING_NFNET:
        image_embeddings1 = np.loadtxt('../input/shopee-price-match-guarantee-embeddings/tf_efficientnet_b5_ns.csv', delimiter=',')
        np.savetxt('eca_nfnet_l0_b24_15.csv', image_embeddings3, delimiter=',')
        image_embeddings4 = np.loadtxt('../input/shopee-price-match-guarantee-embeddings/efficientnet_b3.csv', delimiter=',')
else:
    if EFF_B5:
        image_embeddings1 = np.loadtxt('../input/shopee-price-match-guarantee-embeddings/tf_efficientnet_b5_ns.csv', delimiter=',')
    image_embeddings3 = np.loadtxt('../input/shopee-price-match-guarantee-embeddings/eca_nfnet_l0_b24_15.csv', delimiter=',')
    image_embeddings4 = np.loadtxt('../input/shopee-price-match-guarantee-embeddings/efficientnet_b3.csv', delimiter=',')

Building Model Backbone for eca_nfnet_l0 model


/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 6900/6900 [56:23<00:00,  2.04it/s]


image embeddings3 shape is (138000, 512)
Building Model Backbone for efficientnet_b3 model


100%|██████████| 6900/6900 [54:05<00:00,  2.13it/s]


Our image embeddings shape is (20, 512)
Building Model Backbone for eca_nfnet_l0 model


100%|██████████| 6900/6900 [56:01<00:00,  2.05it/s]


Our image embeddings shape is (20, 512)
Building Model Backbone for eca_nfnet_l0 model


100%|██████████| 6900/6900 [56:30<00:00,  2.04it/s]


Our image embeddings shape is (20, 512)


In [19]:
image_embeddings_other = image_embeddings3 / 3 + image_embeddings4 / 3  + image_embeddings5 / 3
image_embeddings_clothes = image_embeddings3 / 3 + image_embeddings4 / 3  + image_embeddings6 / 3
del image_embeddings4,image_embeddings5,image_embeddings6

In [20]:
other_emb = pd.DataFrame(image_embeddings_other).to_csv('other_emb.csv', index = False)
del other_emb
clothes_emb = pd.DataFrame(image_embeddings_other).to_csv('clothes_emb.csv', index = False)
del clothes_emb

In [21]:
image_predictions_other = get_image_predictions_other(df, image_embeddings1, image_embeddings3, image_embeddings_other, 
                                          predictions34_th=EMBEDDING34_TH)
image_predictions_clothes = get_image_predictions_clothes(df, image_embeddings1, image_embeddings3, image_embeddings_clothes, 
                                          predictions34_th=EMBEDDING34_TH)



100%|██████████| 41515/41515 [00:42<00:00, 980.26it/s] 


In [22]:
image_predictions_other.extend(image_predictions_clothes)

In [23]:
image_predictions = image_predictions_other.copy()
del image_predictions_other, image_predictions_clothes

In [24]:
df['similar'] = image_predictions

df1 = pd.read_csv('../input/shoppe-web-infer-csv/test.csv')    
df1 = df1['posting_id'] 
df = pd.merge(df1, df, on = 'posting_id', how = 'left')
df = df.rename(columns={'posting_id':'item_id'})
df[['item_id', 'similar']].to_csv('similar.csv', index = False)
df[['item_id', 'similar']].head()

,item_id,similar
0,girls_clothes0001,"[girls_clothes0001, girls_clothes2609, girls_c..."
1,girls_clothes0002,"[girls_clothes0002, girls_clothes4255, girls_c..."
2,girls_clothes0003,"[girls_clothes0003, girls_clothes2800, girls_c..."
3,girls_clothes0004,"[girls_clothes0004, home_life3077, beauty_care..."
4,girls_clothes0005,"[girls_clothes0005, cultural_and_creative_prod..."
